[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alessandronascimento/BioMolComp/blob/main/P08/Pratica08B.ipynb)

# Pratica 08 - Parte II:  #

## Representação Gráfica de Genomas com Biopython ##

Uma parte importante da análise de genomas envolve a definição da existência de genes e sua representação dentro do genoma. A identificação de genes (particularmente em eucariotos) pode ser feita com ferramentas como o [GeneID](https://genome.crg.es/geneid.html), por exemplo que prevê a existência de uma ORF e de exons e íntrons.

Já a representação dos genes identificados pode ser feita por ferramentas como o **Genome Diagram**, presente no Biopython. Nesta parte da prática faremos a análise de três genomas usando estas ferramentas. 

## Objetivo: ##

Representar graficamente os genomas do vírus *SARS-COV2*, e dos procariotos *Escherichia coli* e *Haemophilus influenzae*.

## Roteiro ##

Faça a análise gráfica dos genomas destes organismos usando os códigos de acesso abaixo. Os códigos abaixo permitem fazer esta análise de forma simples.



*   SARS-Cov2: NC_045512.2
*   H*aemophilus influenzae*: CP007470.1
*   *E. coli*: U00096.3



In [ ]:
#@title Preparando o Ambiente para a Execução

!pip3 install biopython
!pip3 install reportlab
!pip3 install Pillow

from reportlab.lib import colors
from reportlab.lib.units import cm
from Bio.Graphics import GenomeDiagram
from Bio import SeqIO
from Bio.SeqFeature import SeqFeature, FeatureLocation

from Bio.Seq import Seq
from Bio import Entrez

from IPython.display import Image

from google.colab import auth
auth.authenticate_user()

import requests

gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
#print(gcloud_tokeninfo['email'])
user_email = gcloud_tokeninfo['email']
print("User email %s will be used to autenticate Entrez access." % user_email)


In [68]:
entry = "" #@param {type:"string"}

In [ ]:
Entrez.email = user_email
handle = Entrez.efetch(db="nucleotide", id=entry, rettype="gb", retmode="text")
record = SeqIO.read(handle, "genbank")
print(record)
handle.close()

gd_diagram = GenomeDiagram.Diagram(record.id)
gd_track_for_features = gd_diagram.new_track(1, name="Annotated Features")
gd_feature_set = gd_track_for_features.new_set()

for feature in record.features:
    if feature.type != "gene":
        # Exclude this feature
        continue
    if len(gd_feature_set) % 2 == 0:
        color = colors.blue
    else:
        color = colors.lightblue
    gd_feature_set.add_feature(
        feature, sigil="ARROW", color=color, label=True, label_size=6, label_angle=0
    )

# I want to include some strandless features, so for an example
# will use EcoRI recognition sites etc.
for site, name, color in [
#    ("GAATTC", "EcoRI", colors.green),
#    ("CCCGGG", "SmaI", colors.orange),
#    ("AAGCTT", "HindIII", colors.red),
#    ("GGATCC", "BamHI", colors.purple),
]:
    index = 0
    while True:
        index = record.seq.find(site, start=index)
        if index == -1:
            break
        feature = SeqFeature(FeatureLocation(index, index + len(site)))
        gd_feature_set.add_feature(
            feature,
            color=color,
            name=name,
            label=True,
            label_size=10,
            label_color=color,
        )
        index += len(site)

gd_diagram.draw(format="linear", pagesize="A4", fragments=15, start=0, end=len(record))
gd_diagram.write("genome_linear.png", "PNG")


gd_diagram.draw(
    format="circular",
    circular=True,
    pagesize=(20 * cm, 20 * cm),
    start=0,
    end=len(record),
    circle_core=0.5,
)
gd_diagram.write("genome_circular.png", "PNG")

In [ ]:
Image('genome_linear.png')

In [ ]:
Image('genome_circular.png')